# 📦 Prepare `dataset/` at repo root (robust, repo-root aware)

This notebook downloads **dataset.zip** from Google Drive, extracts it to  
`<repo>/dataset/` (`soccer_vision/dataset`), fixes any nested paths created by the
notebook’s working directory, and validates the expected structure:

```

soccer_vision/
├── dataset/
    ├──train/
        ├── images/
        ├── labels/
    ├── val/           (optional; can be created later by the training script)
```

## Install/imports

In [1]:
import os, zipfile, shutil, sys
from pathlib import Path

# Install gdown if needed
try:
    import gdown  # noqa: F401
except Exception:
    %pip -q install gdown
import gdown

## Configuration

- `ROOT_NAME`: the folder name of your repo.
- `GDRIVE_DATASET_ID`: the Google Drive **file id** for `dataset.zip`.

In [2]:
ROOT_NAME = "soccer_vision"  # repo folder name
GDRIVE_DATASET_ID = "13c8_NnjszJBDuLAHCr8jKq3eBP7Pocez"  # <-- your dataset.zip file id

## Helper functions

- `resolve_repo_root()` finds the repo root no matter where this notebook runs.
- `dataset_looks_ok()` checks expected folders.
- `tree()` prints a small directory tree for quick inspection.

In [3]:
def resolve_repo_root() -> Path:
    """Find the repo root named ROOT_NAME, regardless of where this notebook runs."""
    env = os.getenv("SOCCER_VISION_ROOT")
    if env:
        p = Path(env).expanduser().resolve()
        if p.name != ROOT_NAME:
            print(f"[WARN] SOCCER_VISION_ROOT is set to {p}, but folder name != {ROOT_NAME}")
        return p

    here = Path.cwd().resolve()
    if here.name == ROOT_NAME and (here / "notebooks").exists():
        return here
    if here.name == "notebooks" and here.parent.name == ROOT_NAME:
        return here.parent
    for p in [here, *here.parents]:
        if p.name == ROOT_NAME:
            return p
    for p in [here, *here.parents]:
        if (p / "notebooks").exists():
            return p
    return here

def dataset_looks_ok(ds: Path) -> bool:
    return (ds / "train" / "images").exists() and (ds / "train" / "labels").exists()

def tree(path: Path, max_files=5):
    print(f"\n[Tree] {path.resolve()}")
    for root, dirs, files in os.walk(path):
        indent = "  " * (len(Path(root).relative_to(path).parts))
        print(f"{indent}{Path(root).name}/")
        for d in sorted(dirs):
            print(f"{indent}  {d}/")
        for i, f in enumerate(sorted(files)):
            if i >= max_files:
                print(f"{indent}  ... (+{len(files)-max_files} more)")
                break
            print(f"{indent}  {f}")


## Locate repo root & define key paths

In [4]:
from soccer_vision.notebooks.modules.paths import find_repo_root

BASE = find_repo_root()
print(f"[BASE] {BASE}")

DATASET_DIR = BASE / "dataset"       # expected final location
DATASET_ZIP = BASE / "dataset.zip"   # temporary zip path

[BASE] Z:\Proyectos\Robotica\footbot\soccer_vision


## Fix a previously nested location

If a prior run placed the dataset under `notebooks/soccer_vision/dataset`, this
moves it back to `<repo>/dataset`.

In [5]:
nested = BASE / "notebooks" / ROOT_NAME / "dataset"
if nested.exists() and not DATASET_DIR.exists():
    print(f"[FIX] Moving nested dataset -> {DATASET_DIR}")
    DATASET_DIR.parent.mkdir(parents=True, exist_ok=True)
    shutil.move(str(nested), str(DATASET_DIR))
    # Attempt to remove the now-empty wrapper dir
    try:
        shutil.rmtree((BASE / "notebooks" / ROOT_NAME), ignore_errors=True)
    except Exception as e:
        print("[WARN] Could not remove nested wrapper folder:", e)

## Download `dataset.zip` from Google Drive (only if missing)

Requires the file to be shared as **Anyone with the link**.

In [6]:
if DATASET_DIR.exists() and dataset_looks_ok(DATASET_DIR):
    print("[INFO] dataset/ already exists, skipping download.")
else:
    print("[DL] Downloading dataset.zip …")
    url = f"https://drive.google.com/uc?id={GDRIVE_DATASET_ID}"
    ok = gdown.download(url, str(DATASET_ZIP), quiet=False)
    if not ok:
        sys.exit("[ERR] gdown failed. Make sure the Drive file is public (Anyone with the link).")

[INFO] dataset/ already exists, skipping download.


## Unzip & normalize to `<repo>/dataset`

In [7]:
if not (DATASET_DIR.exists() and dataset_looks_ok(DATASET_DIR)):
    print("[UNZIP] Extracting …")
    with zipfile.ZipFile(DATASET_ZIP, "r") as z:
        z.extractall(BASE)

    if not dataset_looks_ok(DATASET_DIR):
        # Find a folder that contains train/images and train/labels and move it to dataset/
        candidates = []
        for p in BASE.iterdir():
            if p.is_dir() and (p / "train" / "images").exists() and (p / "train" / "labels").exists():
                candidates.append(p)
        if not candidates:
            sys.exit("[ERR] Could not locate a folder with train/images and train/labels after unzip.")

        pick = next((c for c in candidates if c.name == "dataset"), candidates[0])
        if pick != DATASET_DIR:
            if DATASET_DIR.exists():
                shutil.rmtree(DATASET_DIR)
            shutil.move(str(pick), str(DATASET_DIR))
    
    print("[UNZIP] Finished")

## Clean up temporary files

In [8]:
print("[CLEAN] Deleting dataset.zip (if present)")
try:
    DATASET_ZIP.unlink()
except FileNotFoundError:
    pass

[CLEAN] Deleting dataset.zip (if present)


## Final validation

We should now have `soccer_vision/dataset/train/images` and `soccer_vision/dataset/train/labels`.

In [9]:
if not dataset_looks_ok(DATASET_DIR):
    sys.exit("[ERR] dataset/ is missing expected subfolders (train/images and train/labels).")

print("[OK] Dataset ready at:", DATASET_DIR)
tree(DATASET_DIR)

[OK] Dataset ready at: Z:\Proyectos\Robotica\footbot\soccer_vision\dataset

[Tree] Z:\Proyectos\Robotica\footbot\soccer_vision\dataset
dataset/
  train/
  val/
  classes.txt
  notes.json
  train/
    images/
    labels/
    labels.cache
    images/
      00b200b7-photo_2025-10-12_16-30-16_aug_2.jpg
      01aef225-photo_2025-10-12_16-30-49_aug_2.jpg
      02147359-photo_2025-10-12_22-01-54_aug_3.jpg
      02260e77-photo_2025-10-12_22-01-58_aug_1.jpg
      0392df9f-photo_2025-10-12_16-28-24_aug_3.jpg
      ... (+455 more)
    labels/
      00b200b7-photo_2025-10-12_16-30-16_aug_2.txt
      01aef225-photo_2025-10-12_16-30-49_aug_2.txt
      02147359-photo_2025-10-12_22-01-54_aug_3.txt
      02260e77-photo_2025-10-12_22-01-58_aug_1.txt
      0392df9f-photo_2025-10-12_16-28-24_aug_3.txt
      ... (+455 more)
  val/
    images/
    labels/
    labels.cache
    images/
      0cd54e92-photo_2025-10-12_22-02-19_aug_1.jpg
      16444e52-photo_2025-10-12_22-01-24_aug_2.jpg
      1b18bdc6-photo_20

## Install Ultralytics (YOLO)
Only needed the first time you run this environment.

In [10]:
%pip -q install ultralytics torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Training parameters
Adjust these, then run the next cell to retrain.


In [11]:
MODEL_BACKBONE   = "yolo11s.pt"
EPOCHS           = 60
IMG_SIZE         = 640
BATCH_SIZE       = 16
DEVICE           = "0"            # GPU id like "0" or "cpu"
TRAIN_PCT        = 0.90           # used only if val/ is empty and --no-split is False
NO_SPLIT         = False          # set True if you don't want main.py to create/move a val split
PATIENCE         = 20             # early stopping patience
WORKERS          = 8              # dataloader workers
SEED             = 0              # for reproducible split & training seed
RUN_NAME         = None           # if None -> defaults to "<backbone_stem>_train"
OUT_SUBDIR       = None           # if None -> defaults to "<backbone_stem>" under soccer_vision/models
FINAL_WEIGHTS    = "soccer_yolo.pt"  # file name saved in models/<OUT_SUBDIR>/
CONF_FOR_VALPREV = 0.25           # quick val prediction confidence


## Retrain the model
This calls the project script so all runs/models land **inside** `soccer_vision/`.

In [12]:
# Make sure we can import notebooks.modules.* no matter the current CWD
from pathlib import Path
import sys

here = Path.cwd().resolve()
repo = here if (here.name == "soccer_vision") else (here.parent if here.name == "notebooks" else None)
if not repo:
    # walk up until we find soccer_vision/
    for p in [here, *here.parents]:
        if (p / "soccer_vision" / "notebooks" / "modules").exists():
            repo = p / "soccer_vision"
            break

assert repo and (repo / "notebooks" / "modules").exists(), f"Could not find modules under {here}"
if str(repo) not in sys.path:
    sys.path.insert(0, str(repo))  # add repo root to sys.path

print("[REPO]", repo)

[REPO] Z:\Proyectos\Robotica\footbot\soccer_vision


In [13]:
# Live logs training
from soccer_vision.notebooks.modules.logging_utils import get_logger, log
from soccer_vision.notebooks.modules.paths import find_repo_root
from soccer_vision.notebooks.modules.train import train_yolo

# Optional: INFO (or DEBUG) level
get_logger()  # already INFO by default

BASE = find_repo_root()
SV_DIR = BASE  # shorthand

# ---- knobs to tweak from notebook ----
MODEL_BACKBONE   = SV_DIR / "yolo11s.pt"   # or "yolo11s.pt" to auto-download
EPOCHS           = 60
IMG_SIZE         = 640
BATCH_SIZE       = 16
DEVICE           = "0"                     # "0" for first GPU; "cpu" to force CPU
WORKERS          = 8
SEED             = 0
PATIENCE         = 20
TRAIN_PCT        = 0.90
COPY_SPLIT       = False
NO_SPLIT         = False
FINAL_WEIGHTS    = "soccer_yolo.pt"
CONF_FOR_VALPREV = 0.25
RUN_NAME         = "yolo11s_train"
OUT_SUBDIR       = "yolo11s"               # models/<OUT_SUBDIR>

result = train_yolo(
    base_dir=SV_DIR,
    model=MODEL_BACKBONE,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE,
    workers=WORKERS,
    seed=SEED,
    patience=PATIENCE,
    train_pct=TRAIN_PCT,
    copy_split=COPY_SPLIT,
    final_name=FINAL_WEIGHTS,
    conf_for_valprev=CONF_FOR_VALPREV,
    run_name=RUN_NAME,
    out_subdir=OUT_SUBDIR,
    no_split=NO_SPLIT,
    logger=log,
)

print("\n--- RESULT ---")
print("Best Weights :", result.best_weights)
print("Run Dir      :", result.run_dir)
print("Artifacts    :", result.artifacts_dir)
print("Val Predicts :", result.val_pred_dir)
print("Device Used  :", result.device_used)


[INFO] [INFO] Validation split already present. Skipping split.
[INFO] [OK] Wrote Z:\Proyectos\Robotica\footbot\soccer_vision\data.yaml
[INFO] [INFO] ULTRALYTICS_HOME=Z:\Proyectos\Robotica\footbot\soccer_vision
[INFO] [TRAIN] model=Z:\Proyectos\Robotica\footbot\soccer_vision\yolo11s.pt epochs=60 imgsz=640 batch=16 device=0 workers=8 seed=0 patience=20
New https://pypi.org/project/ultralytics/8.3.221 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.0  Python-3.11.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=Z:\Proyectos\Robotica\footbot\soccer_vision\yolo11s.pt, data=Z:\Proyectos\Robotica\footbot\soccer_vision\data.yaml, epochs=60, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=Z:\Proyectos\Robotica\footbot\soccer_vision\runs, name=yolo11s_train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, determinis

train: Scanning Z:\Proyectos\Robotica\footbot\soccer_vision\dataset\train\labels.cache... 460 images, 0 backgrounds, 0 corrupt: 100%|██████████| 460/460 [00:00
val: Scanning Z:\Proyectos\Robotica\footbot\soccer_vision\dataset\val\labels.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it


Plotting labels to Z:\Proyectos\Robotica\footbot\soccer_vision\runs\yolo11s_train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to Z:\Proyectos\Robotica\footbot\soccer_vision\runs\yolo11s_train
Starting training for 60 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/60      4.24G      1.016      3.097       1.25         21        640: 100%|██████████| 29/29 [00:24<00:00,  1.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.49s/it]

                   all         52         69      0.727      0.685      0.738      0.484


...

      60/60      4.29G     0.3015     0.1889     0.8272         12        640: 100%|██████████| 29/29 [00:44<00:00,  1.52s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.30s/it]

                   all         52         69      0.999      0.978      0.991      0.919



60 epochs completed in 0.714 hours.
Optimizer stripped from Z:\Proyectos\Robotica\footbot\soccer_vision\runs\yolo11s_train\weights\last.pt, 19.2MB
Optimizer stripped from Z:\Proyectos\Robotica\footbot\soccer_vision\runs\yolo11s_train\weights\best.pt, 19.2MB

Validating Z:\Proyectos\Robotica\footbot\soccer_vision\runs\yolo11s_train\weights\best.pt...
Ultralytics 8.3.0  Python-3.11.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)
YOLO11s summary (fused): 238 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.36it/s]


                   all         52         69      0.999      0.978      0.991      0.918
                  goal         25         25      0.998          1      0.995      0.892
              opponent         43         44          1      0.956      0.987      0.944
Speed: 0.5ms preprocess, 7.9ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to Z:\Proyectos\Robotica\footbot\soccer_vision\runs\yolo11s_train
[INFO] [OK] Saved best weights to Z:\Proyectos\Robotica\footbot\soccer_vision\models\yolo11s\soccer_yolo.pt
[INFO] [OK] Copied training artifacts to Z:\Proyectos\Robotica\footbot\soccer_vision\models\yolo11s\train_artifacts
[INFO] [PREDICT] Running quick predictions on val images…

image 1/52 Z:\Proyectos\Robotica\footbot\soccer_vision\dataset\val\images\0cd54e92-photo_2025-10-12_22-02-19_aug_1.jpg: 640x640 1 goal, 1 opponent, 43.5ms
image 2/52 Z:\Proyectos\Robotica\footbot\soccer_vision\dataset\val\images\16444e52-photo_2025-10-12_22-01-24_aug_2.jpg: 640x640 1 goa

<!-- === Results summary (pretty, compact) === -->

<style>
.results-wrap { font-family: system-ui, -apple-system, Segoe UI, Roboto, "Helvetica Neue", Arial, "Noto Sans", "Liberation Sans", sans-serif; }
.results-head { margin: 0 0 .6rem 0; font-size: 1.35rem; font-weight: 700; }
.results-sub { margin: 0 0 1.2rem 0; color: #6b7280; font-size: .95rem; }
.grid { display: grid; grid-template-columns: repeat(3, 1fr); gap: 10px; }
.grid-2 { display: grid; grid-template-columns: 1.2fr 1fr; gap: 12px; margin-top: 10px; }
.card { background:#0b0b0b0d; border:1px solid #e5e7eb; border-radius:12px; padding:10px; }
.card h4 { margin:.2rem 0 .5rem 0; font-size:.95rem; }
.fig { display:flex; flex-direction:column; align-items:center; gap:.35rem; }
.fig img { width:100%; height:auto; border-radius:8px; border:1px solid #e5e7eb; }
.caption { color:#6b7280; font-size:.85rem; line-height:1.2; }
ul.tight { margin:.4rem 0 .6rem 1.1rem; padding:0; }
ul.tight li { margin:.2rem 0; }
@media (max-width: 980px) {
  .grid { grid-template-columns: 1fr; }
  .grid-2 { grid-template-columns: 1fr; }
}
</style>

<div class="results-wrap">

<div class="results-head">✅ Training Results — Quick Visual Summary</div>
<div class="results-sub">Figures generated by Ultralytics and saved to <code>soccer_vision/results/</code>.</div>

<div class="grid">

<div class="card fig">
  <h4>F1–Confidence Curve</h4>
  <img src="../results/F1_curve.png" alt="F1 curve">
  <div class="caption">
    • F1 balances precision & recall as the confidence threshold changes.<br>
    • Use the peak as a good default threshold (in our run ≈ <b>0.86</b>).
  </div>
</div>

<div class="card fig">
  <h4>Precision–Recall (PR) Curve</h4>
  <img src="../results/PR_curve.png" alt="PR curve">
  <div class="caption">
    • Area near top-right means strong performance.<br>
    • Legend shows per-class AP and overall mAP@0.5.
  </div>
</div>

<div class="card fig">
  <h4>Recall–Confidence Curve</h4>
  <img src="../results/R_curve.png" alt="R curve">
  <div class="caption">
    • Recall starts high and drops as threshold increases.<br>
    • Explains why F1 peaks before very high thresholds.
  </div>
</div>

</div>

<div class="grid-2">

<div class="card fig">
  <h4>Normalized Confusion Matrix</h4>
  <img src="../results/confusion_matrix_normalized.png" alt="Confusion matrix normalized">
  <div class="caption">
    • Rows = True class, Columns = Predicted class (each row sums to 1).<br>
    • Diagonal cells are correct predictions (higher is better).<br>
    • Off-diagonals show typical confusions (e.g., <i>opponent → background</i>).
  </div>
</div>

<div class="card">
  <h4>How to read <code>results.png</code></h4>
  <div class="fig"><img src="../results/results.png" alt="Training curves grid"></div>
  <ul class="caption tight">
    <li><b>train/box_loss, train/cls_loss, train/dfl_loss</b> — training losses trending down = learning.</li>
    <li><b>val/box_loss, val/cls_loss, val/dfl_loss</b> — validation losses; should generally follow train (no big divergence).</li>
    <li><b>metrics/precision(B), metrics/recall(B)</b> — quality on validation set improves as they rise toward 1.0.</li>
    <li><b>metrics/mAP50(B)</b> & <b>mAP50-95(B)</b> — summary accuracy; smooth upward trend is ideal.</li>
  </ul>
  <div class="caption">
    If val metrics drop while train improves, consider more augmentation, earlier stopping, or smaller LR.
  </div>
</div>

</div>

<div class="card" style="margin-top:12px;">
  <h4>Takeaways & Tips</h4>
  <ul class="tight">
    <li><b>Default threshold:</b> start around the F1 peak (≈ <b>0.86</b>) and tune per use-case.</li>
    <li><b>Where are files?</b> <code>soccer_vision/results/</code> (or <code>models/&lt;subdir&gt;/train_artifacts/</code>).</li>
    <li><b>Best weights:</b> <code>models/&lt;subdir&gt;/soccer_yolo.pt</code> for inference.</li>
  </ul>
</div>

</div>
